In [37]:
import pandas as pd
import re

In [58]:
notesData = pd.read_csv("data/notes.tsv", sep="\t")
noteStatusData = pd.read_csv("data/noteStatusHistory.tsv", sep="\t")
print(notesData.shape)
print(noteStatusData.shape)

C:\Users\Chico\AppData\Local\Temp\ipykernel_11576\57774694.py:1: DtypeWarning: Columns (5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  notesData = pd.read_csv("data/notes.tsv", sep="\t")
C:\Users\Chico\AppData\Local\Temp\ipykernel_11576\57774694.py:2: DtypeWarning: Columns (10,19) have mixed types. Specify dtype option on import or set low_memory=False.
  noteStatusData = pd.read_csv("data/noteStatusHistory.tsv", sep="\t")


(1537510, 23)
(1674962, 23)


## Creating data subset for annotation

In [68]:
annoNotes = pd.DataFrame(columns=["noteId", "summary", "label1", "label2"])
misCount = 0
nMisCount = 0
index = 0

for i in range(1000):

    noteId = notesData.iloc[-i]["noteId"]
    classification = notesData.iloc[-i]["classification"]
    believable = notesData.iloc[-i]["believable"]
    harmful = notesData.iloc[-i]["harmful"]
    valDiff = notesData.iloc[-i]["validationDifficulty"]
    summary = notesData.iloc[-i]["summary"]
    
    noteStatus = noteStatusData[noteStatusData["noteId"] == noteId]
    createdAt = noteStatus["createdAtMillis"]
    currentStatusTime = noteStatus["timestampMillisOfCurrentStatus"]
    currentStatus = noteStatus["currentStatus"]

    print(currentStatus[0])


    if (classification == "MISINFORMED_OR_POTENTIALLY_MISLEADING" and # We are looking for notes that are supposed to be shown under a tweet
        believable not in ["BELIEVABLEBYFEW", "BELIEVABLE_BY_MANY"] and # We don't want notes that use depricated metrics
        harmful not in ["LITTLE*HARM", "CONSIDERABLE*HARM"] and 
        valDiff not in ["EASY", "CHALLENGING"] and
        "http" in summary and # We want notes that have a source in them
        currentStatus != "NEEDS_MORE_RATINGS" and # We want a note that is either rated "helpful" or "not helpful"
        (currentStatusTime < 1735437720000)): # We want the rating to be at least a day old

        misCount += 1
        annoNotes.loc[index, "noteId"] = noteId
        annoNotes.loc[index, "summary"] = summary
        index += 1

    if index >= 201:
        break


annoNotes.to_csv("data/annotatation.csv")

KeyError: 0